# Dokumentation der Hautkrebs-Erkennungs-API

## Einführung

Diese API ermöglicht die Erkennung von Hautkrebs mithilfe eines Machine-Learning-Modells, das auf Bildern von Hautläsionen basiert. Sie verwendet **FastAPI**, **PyTorch** und **EfficientNet**, um Bilder und tabellarische Daten (Alter und Geschlecht) zu verarbeiten und präzise Diagnosen zu generieren.

## Projekteinrichtung

### Abhängigkeiten

Stellen Sie sicher, dass die erforderlichen Pakete installiert sind, bevor Sie die API ausführen:
```bash
pip install fastapi uvicorn torch torchvision joblib numpy pillow
```

### Erforderliche Dateien

- `model.py`: Enthält die Implementierung des Modells `SkinCancerModel`.
- `models/best_model.pth`: Gewichte des vortrainierten Modells.
- `models/scaler.pkl`: Skalierer zur Normalisierung der tabellarischen Daten.

## Endpunkte

### 1. `/predict/` - Diagnose von Hautkrebs durchführen

**Methode:** `POST`

**Parameter:**
- `file` (JPG/PNG-Bild): Bild der Hautläsion.
- `age` (Ganzzahl): Alter des Patienten (Jahre).
- `sex` (String): Geschlecht des Patienten (`male` oder `female`).

**Beispiel für die Verwendung:**
```bash
curl -X 'POST' \
  'http://localhost:8000/predict/' \
  -F 'file=@lesion.jpg' \
  -F 'age=45' \
  -F 'sex=male'
```

**Antwort:**
```json
{
  "diagnosis": "Benign",
  "confidence": 92.5
}
```

---

### 2. `/result/` - Letztes Vorhersageergebnis abrufen

**Methode:** `GET`

**Beispiel für die Verwendung:**
```bash
curl -X 'GET' 'http://localhost:8000/result/'
```

**Antwort:**
```json
{
  "age": 45,
  "sex": "male",
  "diagnosis": "Benign",
  "confidence": 92.5
}
```

---

### 3. `/health/` - Status der API überprüfen

**Methode:** `GET`

**Beispiel für die Verwendung:**
```bash
curl -X 'GET' 'http://localhost:8000/health'
```

**Antwort:**
```json
{
  "status": "ok",
  "device": "cuda"
}
```

## Arbeitsablauf

1. Der Benutzer sendet ein Bild der Läsion zusammen mit Alter und Geschlecht.
2. Die API validiert das Bild und die Daten.
3. Das Bild wird mit **EfficientNet** verarbeitet, und die tabellarischen Daten werden mit einem **MLP** verarbeitet.
4. Es wird eine Vorhersage (`Benign` oder `Malignant (melanoma)`) mit dem Konfidenzniveau generiert.
5. Der Benutzer kann das zuletzt gespeicherte Ergebnis abfragen.

## Sicherheitsüberlegungen

- Es wird empfohlen, `allow_origins=["*"]` in der Produktion auf eine restriktivere Liste zu ändern.
- Bildformate und Werte für Alter/Geschlecht werden validiert, um Fehler zu vermeiden.
- Globale Fehlerbehandlung verbessert die Stabilität der API.

## Bereitstellung

Um die API lokal auszuführen:
```bash
uvicorn main:app --host 0.0.0.0 --port 8000
```

Für die Bereitstellung in der Produktion sollten Sie **Docker** oder **Gunicorn** mit Uvicorn verwenden:
```bash
gunicorn -w 4 -k uvicorn.workers.UvicornWorker main:app
```

---

**Hinweis**: Diese Dokumentation ist als lebendes Dokument gedacht. Wenn sich die API weiterentwickelt, werden Aktualisierungen vorgenommen, um neue Funktionen, Verbesserungen und Änderungen in den Best Practices widerzuspiegeln.